## Authentication to Spotify API

### Imports

In [1]:
import pandas as pd
import requests
import base64
import json
import os

### Request for access token

In [2]:
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

In [3]:
auth_string = client_id + ":" + client_secret
auth_bytes = auth_string.encode("utf-8")
auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

In [4]:
url = "https://accounts.spotify.com/api/token"
headers = {
    "Authorization": "Basic " + auth_base64,
    "Content-type": "application/x-www-form-urlencoded"
}
data = {"grant_type": "client_credentials"}

In [68]:
result = requests.post(url, headers=headers, data=data)
result

<Response [200]>

In [69]:
json_result = json.loads(result.content)
token = json_result["access_token"]
token

'BQDUA2Sk8hu8O4hWKBUgBUpLgTB5T1ZmciACBt9VlnkmxKeTUVAOZYx3BYzHin6EQ2Kg0wnxaMqT3MeIv5s0PfvgG3wGOM5ZClMWtFG3VXUIaRwjpbg'

In [63]:
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}
# get_auth_header(token)

### Searching for artist

In [64]:
def search_artist(token, artist_name):
    search_url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=2"
    
    query_url = search_url + query
    result = requests.get(query_url, headers=headers)
    json_result = json.loads(result.content)
    
    return json_result


In [70]:
# search_artist(token, "Kanye West")

### Getting the id

In [10]:
def get_artist_id(token, artist_name):
    search_url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=2"
    
    query_url = search_url + query
    result = requests.get(query_url, headers=headers)
    json_result = json.loads(result.content)["artists"]["items"]
    
    artist_id = json_result[0]
    
    if len(json_result) == 0:
        return f"No artists with this name"
    
    return artist_id["id"]

In [11]:
get_artist_id(token, "Kanye West")

'5K4W6rqBFWDnAN6FQUkS6x'

### Getting the artist name

In [57]:
def get_artist_name(token, id):
    search_url = f"https://api.spotify.com/v1/artists/{id}"
    headers = get_auth_header(token)
    
    result = requests.get(search_url, headers=headers)
    json_result = json.loads(result.content)["name"]
    
    return json_result

In [58]:
test = get_artist_name(token, "5K4W6rqBFWDnAN6FQUkS6x")
test

'Kanye West'

### Getting the related artists

In [71]:
def get_related_artists(token, id):
    artists_url = f"https://api.spotify.com/v1/artists/{id}/related-artists"
    headers = get_auth_header(token)
    
    result = requests.get(artists_url, headers=headers)
    json_result = json.loads(result.content)["artists"]
    
    return json_result

In [75]:
artists = get_related_artists(token, "3nFkdlSjzX9mRTtwJOzDYB")
# artists
# len(artists)

### Automating the process

In [115]:
related_artist = {}
related_artist["main_artist"] = []
related_artist["id"] = []
related_artist["name"] = []
related_artist["popularity"] = []
related_artist["genres"] = []

In [116]:
def collect_related_artists(token, artist_name):
    related_artist["main_artist"].append(artist_name)
    main_artist_id = get_artist_id(token, artist_name)
    artists = get_related_artists(token, main_artist_id)
    
    for i in range(len(artists)):
        related_artist["id"].append(artists[i]["id"])
        related_artist["name"].append(artists[i]["name"])
        related_artist["popularity"].append(artists[i]["popularity"])
        related_artist["genres"].append(artists[i]["genres"])
    
    data_trash = related_artist.copy()
    aux = 0
    while (len(data_trash["id"]) != 0):
        related_artist["main_artist"].append(get_artist_name(token, artists[aux]["id"]))
        # print(aux)
        side_artists = get_related_artists(token, artists[aux]["id"])
        
        for i in range(len(side_artists)):
            related_artist["id"].append(side_artists[i]["id"])
            related_artist["name"].append(side_artists[i]["name"])
            related_artist["popularity"].append(side_artists[i]["popularity"])
            related_artist["genres"].append(side_artists[i]["genres"])
        
        data_trash["id"].pop(0)
        aux += 1
        if aux == 20:
            return related_artist
    
        

In [117]:
data = collect_related_artists(token, "Kanye West")

In [118]:
data

{'main_artist': ['Kanye West',
  'JAY-Z',
  'Kid Cudi',
  'Kendrick Lamar',
  'A$AP Rocky',
  'Baby Keem',
  'Tyler, The Creator',
  'Frank Ocean',
  'Playboi Carti',
  'J. Cole',
  'Childish Gambino',
  '21 Savage',
  'Joey Bada$$',
  'Metro Boomin',
  'Future',
  'Chance the Rapper',
  'Mac Miller',
  'Travis Scott',
  'Lil Uzi Vert',
  'Lil Wayne',
  'Pusha T'],
 'id': ['20qISvAhX20dpIbOOzGK3q',
  '0ONHkAv9pCAFxb0zJwDNTy',
  '4OBJLual30L7gRl5UkeRcT',
  '1sBkRIssrMs1AbVkOJbc7a',
  '59wfkuBoNyhDMQGCljbUbA',
  '5me0Irg2ANcsgc93uaYrpb',
  '55Aa2cqylxrFIXC767Z865',
  '3ipn9JLAPI5GUEo4y4jcoi',
  '1YfEcTuGvBQ8xSD1f53UnK',
  '3ScY9CQxNLQei8Umvpx5g6',
  '5K4W6rqBFWDnAN6FQUkS6x',
  '01QTIT5P1pFP3QnnFSdsJf',
  '1G9G7WwrXka3Z1r7aIDjI7',
  '0NbfKEOTQCcwd6o7wSDOHI',
  '0fA0VVWsXO9YnASrzqfmYu',
  '6O2zJ0tId7g07yzHtX0yap',
  '17lzZA2AlOHwCwFALHttmp',
  '5pnbUBPifNnlusY8kTBivi',
  '0Mz5XE0kb1GBnbLQm2VbcO',
  '0YWxKQj2Go9CGHCp77UOyy',
  '2hPgGN4uhvXAxiXQBIXOmE',
  '5IcR3N7QB1j6KBL8eImZ8m',
  '2P5sC9c

In [114]:
data["name"]

['JAY-Z',
 'Kid Cudi',
 'Kendrick Lamar',
 'A$AP Rocky',
 'Baby Keem',
 'Tyler, The Creator',
 'Frank Ocean',
 'Playboi Carti',
 'J. Cole',
 'Childish Gambino',
 '21 Savage',
 'Joey Bada$$',
 'Metro Boomin',
 'Future',
 'Chance the Rapper',
 'Mac Miller',
 'Travis Scott',
 'Lil Uzi Vert',
 'Lil Wayne',
 'Pusha T',
 'KIDS SEE GHOSTS',
 'ScHoolboy Q',
 'Joey Bada$$',
 'Lupe Fiasco',
 'Pusha T',
 'Denzel Curry',
 'Chance the Rapper',
 'A$AP Rocky',
 'Earl Sweatshirt',
 'Vince Staples',
 'Mac Miller',
 'A$AP Ferg',
 'Baby Keem',
 'Isaiah Rashad',
 'A$AP Mob',
 'JID',
 'Childish Gambino',
 'Aminé',
 'Odd Future',
 'BROCKHAMPTON',
 'A$AP Rocky',
 'J. Cole',
 'Tyler, The Creator',
 'Baby Keem',
 'Childish Gambino',
 'Kanye West',
 'Joey Bada$$',
 'ScHoolboy Q',
 'Frank Ocean',
 'Kid Cudi',
 'Chance the Rapper',
 'Mac Miller',
 '21 Savage',
 'Future',
 'Big Sean',
 'Metro Boomin',
 'Playboi Carti',
 'JAY-Z',
 'Migos',
 'JID',
 'A$AP Ferg',
 'A$AP Mob',
 'ScHoolboy Q',
 'Joey Bada$$',
 'Pusha T